In [193]:
%%writefile mapper3.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode
def is_name(word):
    letters = tuple(filter(lambda char: char.isalpha(), word))
    if not letters:
        return False
    return letters[0].isupper() and ''.join(letters[1:]).islower()

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if len(word) < 2:
            continue
        if not bool(word) or word[0].isdigit():
            continue
        if is_name(word):
            print("%s\t%d\t%d" % (word.lower(), 1, 1))
        else:    
            print("%s\t%d\t%d" % (word.lower(), 0, 1))
        #print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        #print "%s\t%d" % (word.lower(), 1)

Overwriting mapper3.py


In [194]:
%%writefile reducer3.py

import sys

current_word = None
cap_count, total_count = 0, 0
for line in sys.stdin:
    try:
        word, cap, count = line.strip().split('\t', 2)
        cap = int(cap)
        count = float(count)
    except ValueError as e:
        continue
    if word == current_word:
        cap_count += cap
        total_count += count
    else:
        if current_word:
            print("%s\t%d\t%f" % (current_word, cap_count, float(cap_count/total_count*100)))
        total_count = count
        cap_count = cap
        current_word = word

if current_word:
    print("%s\t%d\t%f" % (current_word, cap_count, float(cap_count/total_count*100)))

Overwriting reducer3.py


In [195]:
%%writefile mapper3a.py

import sys

for line in sys.stdin:
    try:
        word, cap, count = line.strip().split('\t', 2)
        cap = int(cap)
        count = float(count)
    except ValueError as e:
        continue
    #cap_ratio = float(cap / count * 100);
    #print("---%s\t%d\t%d\t%f" % (word, cap, count, cap_ratio))
    if count > 99.5:
        print("%s\t%d" % (word, cap))

Overwriting mapper3a.py


In [196]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [198]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=2

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Name Count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper3.py,reducer3.py \
    -mapper "python mapper3.py" \
    -reducer "python reducer3.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 
    
OUT_DIR1="names_result_"$(date +"%s%6N")

hdfs dfs -rm -r -skipTrash ${OUT_DIR1} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Name Count" \
    -D mapreduce.job.reduces=0 \
    -files mapper3a.py \
    -mapper "python mapper3a.py" \
    -input ${OUT_DIR}/part* \
    -output ${OUT_DIR1} > /dev/null 
    
hdfs dfs -cat ${OUT_DIR1}/part* | sort -k2nr,2 | head -5 | tail -1    


french	5742


rm: `wordcount_result_1530318537729389': No such file or directory
18/06/30 00:29:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/06/30 00:29:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/06/30 00:29:07 INFO mapred.FileInputFormat: Total input files to process : 1
18/06/30 00:29:08 INFO mapreduce.JobSubmitter: number of splits:2
18/06/30 00:29:09 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1530239586232_0089
18/06/30 00:29:09 INFO impl.YarnClientImpl: Submitted application application_1530239586232_0089
18/06/30 00:29:09 INFO mapreduce.Job: The url to track the job: http://1ac3f7e951bc:8088/proxy/application_1530239586232_0089/
18/06/30 00:29:09 INFO mapreduce.Job: Running job: job_1530239586232_0089
18/06/30 00:29:22 INFO mapreduce.Job: Job job_1530239586232_0089 running in uber mode : false
18/06/30 00:29:22 INFO mapreduce.Job:  map 0% reduce 0%
18/06/30 00:30:28 INFO mapreduce.Job:  map 5% reduce 0%
18/06/30 00:30